# Tool Calling 201: Llama to find Differences between two papers

The image below illustrates the demo in this notebook. 

**Goal:** Use `Meta-Llama-3.1-70b` model to find the differences between two papers

- Step 1: Take the user input query 

- Step 2: Perform an internet search using `tavily` API to fetch the arxiv ID(s) based on the user query

Note: `3.1` models support `brave_search` but this notebook is also aimed at showcasing custom tools. 

The above is important because many-times the user-query is different from the paper name and arxiv ID-this will help us with the next step

- Step 3: Use the web results to extract the arxiv ID(s) of the papers

We will use an 8b model here because who wants to deal with complex regex, that's the main-use case of LLM(s), isn't it? :D

- Step 4: Use `arxiv` API to download the PDF(s) of the papers in user query

- Step 5: For ease, we will extract first 80k words from the PDF and write these to a `.txt` file that we can summarise

- Step 6: Use instances of `Meta-Llama-3.1-8b` instances to summaries the two PDF(s)

- Step 7: Prompt the `70b` model to get the differences between the two papers being discussed

## Part 1: Defining the pieces

We will start by describing all the modules from the image above, to make sure our logic works.

In second half of the notebook, we will write a simple function to take care of the function calling logic

#### Install necessary libraries

In [1]:
!pip3 install groq
!pip3 install arxiv
!pip3 install tavily-python
!pip3 install llama-toolchain
!pip3 install PyPDF2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=1a4536faa8f7c7565467a6b19a85ab27d1759480bab20e206b42f73f857766fc
  Stored in directory: /Users/orangecloud/Library/Caches/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


#### Necessary imports

##### Note: PLEASE REPLACE API KEYS BELOW WITH YOUR REAL ONES

In [2]:
import os, arxiv, PyPDF2
from tavily import TavilyClient
from groq import Groq

# Create the Groq client
client = Groq(api_key='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')

tavily_client = TavilyClient(api_key='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')


#### Main LLM thread: 

We will use a `MAIN_SYSTEM_PROMPT` and a `main_model_chat_history` to keep track of the discussion, since we are using 4 instances of LLM(s) along with this. 

Note, if you paid attention and notice that the SYSTEM_PROMPT here is different-thanks for reading closely! It's always a great idea to follow the official recommendations. 

However, when it's a matter of writing complex regex, we can bend the rules slightly :D

Note, we will outline the functions here and define them as we go

In [3]:
MAIN_SYSTEM_PROMPT = """
Environment: iPython
Cutting Knowledge Date: December 2023
Today Date: 15 September 2024

# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available

You have access to the following functions:

Use the function 'query_for_two_papers' to: Get the internet query results for the arxiv ID of the two papers user wants to compare
{
  "name": "query_for_two_papers",
  "description": "Internet search the arxiv ID of two papers that user wants to look up",
  "parameters": {
    "paper_1": {
      "param_type": "string",
      "description": "arxiv id of paper_name_1 from user query",
      "required": true
    },
    "paper_2": {
      "param_type": "string",
      "description": "arxiv id of paper_name_2 from user query",
      "required": true
    },
  }
}

Use the function 'get_arxiv_ids' to: Given a dict of websearch queries, use a LLM to return JUST the arxiv ID, which is otherwise harder to extract
{
  "name": "get_arxiv_ids",
  "description": "Use the dictionary returned from query_for_two_papers to ask a LLM to extract the arxiv IDs",
  "parameters": {
    "web_results": {
      "param_type": "dictionary",
      "description": "dictionary of search result for a query from the previous function",
      "required": true
    },
  }
}

Use the function 'process_arxiv_paper' to: Given the arxiv ID from get_arxiv_ids function, return a download txt file of the paper that we can then use for summarising
{
  "name": "process_arxiv_paper",
  "description": "Use arxiv IDs extracted from earlier to be downloaded and saved to txt files",
  "parameters": {
    "arxiv_id": {
      "param_type": "string",
      "description": "arxiv ID of the paper that we want to download and save a txt file of",
      "required": true
    },
  }
}

Use the function 'summarize_text_file' to: Given the txt file name based on the arxiv IDs we are working with from earlier, get a summary of the paper being discussed
{
  "name": "summarize_text_file",
  "description": "Summarise the arxiv paper saved in the txt file",
  "parameters": {
    "file_name": {
      "param_type": "string",
      "description": "Filename to be used to get a summary of",
      "required": true
    },
  }
}

If a you choose to call a function ONLY reply in the following format:
<{start_tag}={function_name}>{parameters}{end_tag}
where

start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`

Here is an example,
<function=example_function_name>{"example_name": "example_value"}</function>

Reminder:
- When user is asking for a question that requires your reasoning, DO NOT USE OR FORCE a function call
- Even if you remember the arxiv ID of papers from input, do not put that in the query_two_papers function call, pass the internet look up query
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- When returning a function call, don't add anything else to your response

"""

In [4]:
main_model_chat_history = [
    {
        "role" : "system",
        "content" : MAIN_SYSTEM_PROMPT
    }
]

#### Define the `model_chat` instance

We will be using this to handle all user input(s)

In [5]:

def model_chat(user_input: str, temperature: int = 0, max_tokens=2048):
    
    main_model_chat_history.append({"role": "user", "content": user_input})
    
    #print(chat_history)
    
    #print("User: ", user_input)
    
    response = client.chat.completions.create(model="llama-3.1-70b-versatile",
                                          messages=main_model_chat_history,
                                          max_tokens=max_tokens,
                                          temperature=temperature)
    
    main_model_chat_history.append({
    "role": "assistant",
    "content": response.choices[0].message.content
    })
    
    
    #print("Assistant:", response.choices[0].message.content)
    
    return response.choices[0].message.content

In [6]:
user_input = """
What are the differences between llama 3.1 and BERT?
"""

output = model_chat(user_input, temperature=1)

In [7]:
print(output)

<function=query_for_two_papers>{"paper_1": "Llama 3.1", "paper_2": "BERT"}</function>


If you remember from `Tool_Calling_101.ipynb`, we need a way to extract and manage tool calling based on the response, the system prompt from earlier makes our lives easier to answer do this later :)

First, let's validate the logic and define all the functions as we go:

#### Tavily API: 

We will use the Tavily API to do a web query for the papers based on the model outputs

In [8]:
def query_for_two_papers(paper_1:str , paper_2: str) -> None :
     return [tavily_client.search(f"arxiv id of {paper_1}"), tavily_client.search(f"arxiv id of {paper_2}")]

In [9]:
search_results = query_for_two_papers("llama 3.1", "BERT")
search_results

[{'query': 'arxiv id of llama 3.1',
  'follow_up_questions': None,
  'answer': None,
  'images': [],
  'results': [{'title': 'TheLlama3HerdofModels - arXiv.org',
    'url': 'https://arxiv.org/pdf/2407.21783',
    'content': 'arXiv:2407.21783v2 [cs.AI] 15 Aug 2024. Finetuned Multilingual Longcontext Tooluse Release ... The model architecture of Llama 3 is illustrated in Figure1. The development of our Llama 3 language modelscomprisestwomainstages:',
    'score': 0.9955835,
    'raw_content': None},
   {'title': 'Introducing Llama 3.1: Our most capable models to date - Meta AI',
    'url': 'https://ai.meta.com/blog/meta-llama-3-1/',
    'content': 'Bringing open intelligence to all, our latest models expand context length to 128K, add support across eight languages, and include Llama 3.1 405B—the first frontier-level open source AI model. Llama 3.1 405B is in a class of its own, with unmatched flexibility, control, and state-of-the-art capabilities that rival the best closed source model

In [10]:
user_input = f"""
Here are the search results for the first paper, extract the arxiv ID {search_results[0]}
"""

output = model_chat(user_input, temperature=1)

In [11]:
print(output)

<function=get_arxiv_ids>{"web_results": {'query': 'arxiv id of llama 3.1', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'TheLlama3HerdofModels - arXiv.org', 'url': 'https://arxiv.org/pdf/2407.21783', 'content': 'arXiv:2407.21783v2 [cs.AI] 15 Aug 2024. Finetuned Multilingual Longcontext Tooluse Release ... The model architecture of Llama 3 is illustrated in Figure1. The development of our Llama 3 language modelscomprisestwomainstages:', 'score': 0.9955835, 'raw_content': None}, {'title': 'Introducing Llama 3.1: Our most capable models to date - Meta AI', 'url': 'https://ai.meta.com/blog/meta-llama-3-1/', 'content': 'Bringing open intelligence to all, our latest models expand context length to 128K, add support across eight languages, and include Llama 3.1 405B—the first frontier-level open source AI model. Llama 3.1 405B is in a class of its own, with unmatched flexibility, control, and state-of-the-art capabilities that rival the best closed source m

In [12]:
user_input = f"""
Here are the search results for the second paper now, extract the arxiv ID {search_results[1]}
"""

output = model_chat(user_input, temperature=1)

In [13]:
print(output)

<function=get_arxiv_ids>{"web_results": {'query': 'arxiv id of BERT', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language ...', 'url': 'https://arxiv.org/abs/1810.04805', 'content': 'BERT is a bidirectional transformer model that pre-trains deep representations from unlabeled text and fine-tunes them for various natural language tasks. The paper introduces BERT and reports its state-of-the-art results on eleven tasks, such as question answering and language inference.', 'score': 0.938124, 'raw_content': None}, {'title': '[2103.11943] BERT: A Review of Applications in Natural Language ...', 'url': 'https://arxiv.org/abs/2103.11943', 'content': 'This paper describes the mechanism, areas of application and comparisons of BERT, a popular deep learning-based language model. It also covers some proprietary models and provides references to original scientific articles.', 'score': 0.6907677, 'raw_

#### Extracting Arxiv IDs: 

At this point, you would know the author is allergic to writing regex. To deal with this, we will simply use an `8b` instance to extract the `arxiv id` from the paper:

In [14]:
def get_arxiv_ids(web_results: dict, temperature: int = 0, max_tokens=512):
    # Initialize chat history with a specific prompt to extract arXiv IDs
    arxiv_id_chat_history = [{"role": "system", "content": "Given this input, give me the arXiv ID of the papers. The input has the query and web results. DO NOT WRITE ANYTHING ELSE IN YOUR RESPONSE: ONLY THE ARXIV ID ONCE, the web search will have it repeated mutliple times, just return the it once and where its actually the arxiv ID"}, {"role": "user", "content": f"Here is the query and results{web_results}"}]

    # Call the model to process the input and extract arXiv IDs
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",  # Adjust the model as necessary
        messages=arxiv_id_chat_history,
        max_tokens=max_tokens,
        temperature=temperature
    )
    
    # Append the assistant's response to the chat history
    arxiv_id_chat_history.append({
        "role": "assistant",
        "content": response.choices[0].message.content
    })
    
    # Return the extracted arXiv IDs
    return response.choices[0].message.content

In [15]:
print(get_arxiv_ids(search_results[0]))
print(get_arxiv_ids(search_results[1]))

2407.21783
1810.04805
2103.11943
2410.24159


#### Downloading the papers and extracting details: 

Llama 3.1 family LLM(s) are great enough to use raw outputs extracted from a PDF and summarise them. However, we are still bound by their (great) 128k context length-to live with this, we will extract just the first 80k words. 

The functions below handle the logic of downloading the PDF(s) and extracting their outputs

In [16]:
# Function to download PDF using arxiv library
def download_pdf(arxiv_id, filename):
    paper = next(arxiv.Client().results(arxiv.Search(id_list=[arxiv_id])))
    paper.download_pdf(filename=filename)

# Function to convert PDF to text
def pdf_to_text(filename):
    with open(filename, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            if page.extract_text():
                text += page.extract_text() + " "
    return text

# Function to truncate text after 80k words
def truncate_text(text, limit=20000):
    words = text.split()
    truncated = ' '.join(words[:limit])
    return truncated

# Main function to process an arXiv ID
def process_arxiv_paper(arxiv_id):
    pdf_filename = f"{arxiv_id}.pdf"
    txt_filename = f"{arxiv_id}.txt"
    
    # Download PDF
    download_pdf(arxiv_id, pdf_filename)
    
    # Convert PDF to text
    text = pdf_to_text(pdf_filename)
    
    # Truncate text
    truncated_text = truncate_text(text)
    
    # Save to txt file
    with open(txt_filename, "w", encoding="utf-8") as file:
        file.write(truncated_text)
    print(f"Processed text saved to {txt_filename}")

# Example usage
arxiv_id = "2407.21783"
process_arxiv_paper(arxiv_id)

arxiv_id = "2103.11943"
process_arxiv_paper(arxiv_id)

Processed text saved to 2407.21783.txt
Processed text saved to 2103.11943.txt


#### Summarising logic: 

We can use a `8b` model instance to summarise our papers:

In [17]:

SUMMARISER_PROMPT = """
Cutting Knowledge Date: December 2023
Today Date: 15 September 2024
You are an expert summariser of research papers, below you will get an input of the text from an arxiv paper and your job is to read it carefully and return a concise summary with some bullet points at the end of some key-takeways from it
"""

def summarize_text_file(file_name: str, temperature: int = 0, max_tokens=2048):
    # Read the content of the file
    with open(file_name, 'r') as file:
        file_content = file.read()
    
    # Initialize chat history
    chat_history = [{"role": "system", "content": f"{SUMMARISER_PROMPT}"}, {"role": "user", "content": f"Text of the paper: {file_content}"}]
    
    # Generate a summary using the model
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",  # You can change the model as needed
        messages=chat_history,
        max_tokens=max_tokens,
        temperature=temperature
    )
    
    # Append the assistant's response to the chat history
    chat_history.append({
        "role": "assistant",
        "content": response.choices[0].message.content
    })
    
    # Return the summary
    return response.choices[0].message.content

In [18]:
paper_1_summary = summarize_text_file("2407.21783.txt")
print(paper_1_summary)

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.1-8b-instant` in organization `org_01hsx8mc6ee93v61gnq0h8gkap` on tokens per minute (TPM): Limit 20000, Requested 33292, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [46]:
paper_2_summary = summarize_text_file("2103.11943.txt")
print(paper_2_summary)


BERT is a novel language representation model developed by researchers at Google AI. It stands for Bidirectional Encoder Representations from Transformers and introduces a new approach to pre-training deep bidirectional representations from unlabeled text. Unlike previous models that looked at text sequences either from left-to-right or combined left-to-right and right-to-left training, BERT is designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers.
The key innovation is the application of bidirectional training of Transformer, a popular attention model, to language modeling. This is achieved through two pre-training tasks: Masked Language Model (MLM) and Next Sentence Prediction (NSP). In MLM, the model attempts to predict masked words in a sentence, allowing it to incorporate context from both directions. NSP trains the model to understand relationships between sentences.
BERT significantly outperformed previous 

In [56]:
user_input = f"""
Here are the summaries of the two papers, look at them closely and tell me the differences of the papers: Paper 1 Summary {paper_1_summary} and Paper 2 Summary {paper_2_summary}
"""

output = model_chat(user_input, temperature=1)

In [57]:
print(output)

The two paper summaries are about different language models: Llama 3 and BERT.

The main differences are:

1. Model Type: Llama 3 is a set of foundation models developed by Meta AI, while BERT is a language representation model developed by researchers at Google AI.
2. Model Architecture: Llama 3 uses a dense Transformer architecture, while BERT uses a bidirectional Transformer architecture.
3. Training Process: Llama 3 involves significant infrastructure improvements to handle large-scale distributed training, while BERT uses pre-training tasks such as Masked Language Model (MLM) and Next Sentence Prediction (NSP).
4. Multimodal Capabilities: Llama 3 explores multimodal capabilities by integrating vision and speech components, while BERT focuses on text-based language understanding.
5. Performance: Both models demonstrate competitive performance on various benchmarks, but Llama 3 shows performance on tasks such as multilingual capabilities, coding, reasoning, and tool usage, while BER

## Part 2: Handle the function calling logic: 

Now that we have validated a MVP, we can write a simple function to handle tool-calling:

In [33]:
def handle_llm_output(llm_output):
    # Check if the output starts with "<function="
    if llm_output.startswith("<function="):
        return extract_details_and_call_function(llm_output)
    else:
        # Output does not start with "<function=", return as is
        return llm_output

def extract_details_and_call_function(input_string):
    # Extract the function name and parameters
    prefix = "<function="
    suffix = "</function>"
    start = input_string.find(prefix) + len(prefix)
    end = input_string.find(suffix)
    function_and_params = input_string[start:end]
    
    # Split to get function name and parameters
    function_name, params_json = function_and_params.split(">{")
    function_name = function_name.strip()
    params_json = "{" + params_json
    
    # Convert parameters to dictionary
    params = json.loads(params_json)
    
    # Call the function dynamically
    function_map = {
        "query_for_two_papers": query_for_two_papers,
        "get_arxiv_id": get_arxiv_ids,
        "process_arxiv_paper": process_arxiv_paper,
        "summarise_text_file": summarize_text_file
    }
    
    if function_name in function_map:
        result = function_map[function_name](**params)
        return result
    else:
        return "Function not found"

# Testing usage
llm_outputs = [
    "<function=query_for_two_papers>{\"paper_1\": \"Llama 3.1\", \"paper_2\": \"BERT\"}</function>",
    "Llama 3.2 models are here too btw!"
]

for output in llm_outputs:
    result = handle_llm_output(output)
    print(result)

[{'query': 'arxiv id of Llama 3.1', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'TheLlama3HerdofModels - arXiv.org', 'url': 'https://arxiv.org/pdf/2407.21783', 'content': 'arXiv:2407.21783v2 [cs.AI] 15 Aug 2024. Finetuned Multilingual Longcontext Tooluse Release ... The model architecture of Llama 3 is illustrated in Figure1. The development of our Llama 3 language modelscomprisestwomainstages:', 'score': 0.9961004, 'raw_content': None}, {'title': '[PDF] The Llama 3 Herd of Models - Semantic Scholar', 'url': 'https://www.semanticscholar.org/paper/The-Llama-3-Herd-of-Models-Dubey-Jauhri/6520557cc3bfd198f960cc8cb6151c3474321bd8', 'content': 'DOI: 10.48550/arXiv.2407.21783 Corpus ID: 271571434; The Llama 3 Herd of Models @article{Dubey2024TheL3, title={The Llama 3 Herd of Models}, author={Abhimanyu Dubey and Abhinav Jauhri and Abhinav Pandey and Abhishek Kadian and Ahmad Al-Dahle and Aiesha Letman and Akhil Mathur and Alan Schelten and Amy Yang and Ang

In [ ]:
#fin